In [1]:
import pandas as pd
import sklearn

import service
from classification.OneRClassifier import OneRClassifier
from classification.NaiveBayesClassifier import NaiveBayesClassifier
from classification.KNeighborsClassifier import KNeighborsClassifier

In [2]:
classifiers = [OneRClassifier(), NaiveBayesClassifier(), KNeighborsClassifier()]  # TODO

#### data_4x

In [3]:
df = pd.read_csv('data/data_4x.csv')
df.head()

,x0,x1,x2,x3,y
0,0,0,1,0,0
1,1,0,0,1,1
2,2,0,1,0,1
3,0,1,0,0,1
4,0,1,1,0,1


In [4]:
X, y = df.drop(columns='y'), df['y']
X.shape, y.shape

((10, 4), (10,))

In [5]:
y.replace(0, 'y0', inplace=True)
y.replace(1, 'y1', inplace=True)
y = list(y)

In [6]:
X_pred = pd.DataFrame({'x0': [2], 'x1': [1], 'x2': [1], 'x3': [1]})

##### OneRClassifier

In [7]:
clf = OneRClassifier()
clf.fit(X, y);

In [8]:
service.OneRClassifier_info(clf)

x[3] == 0: y = y1
x[3] == 1: y = y0


In [9]:
clf.predict(X_pred)

array(['y0'], dtype='<U2')

In [10]:
clf.feature_names_in_

array(['x0', 'x1', 'x2', 'x3'], dtype=object)

Compare with similar scikit-learn model

In [11]:
sk_clf = sklearn.tree.DecisionTreeClassifier(max_depth=1)
sk_clf.fit(X, y);

In [12]:
service.sklearn_DecisionTreeClassifier_info(sk_clf)

x[3] <= 0.5: y = y1
x[3] > 0.5: y = y0


In [13]:
sk_clf.predict(X_pred)

array(['y0'], dtype='<U2')

##### NaiveBayesClassifier

In [14]:
clf = NaiveBayesClassifier()
clf.fit(X, y);

In [15]:
service.NaiveBayesClassifier_info(clf)

class_log_probs_:
-0.916290731874155 -0.5108256237659907 

feature_log_probs_:
y  feat_v
0  0        -0.559616
   1        -1.252763
1  2        -0.810930
   0        -1.098612
   1        -1.504077
Name: count, dtype: float64 y  feat_v
0  0        -0.693147
   1        -0.693147
1  0        -0.693147
   1        -0.693147
Name: count, dtype: float64 y  feat_v
0  1        -0.405465
   0        -1.098612
1  0        -0.693147
   1        -0.693147
Name: count, dtype: float64 y  feat_v
0  1        -0.405465
   0        -1.098612
1  0        -0.287682
   1        -1.386294
Name: count, dtype: float64 

feature_missing_log_probs_:
y
0   -1.945910
1   -2.197225
Name: feat_v, dtype: float64 y
0   -1.791759
1   -2.079442
Name: feat_v, dtype: float64 y
0   -1.791759
1   -2.079442
Name: feat_v, dtype: float64 y
0   -1.791759
1   -2.079442
Name: feat_v, dtype: float64 



In [16]:
clf.predict(X_pred)

array(['y1'], dtype='<U2')

Compare with similar scikit-learn model

In [17]:
sk_clf = sklearn.naive_bayes.CategoricalNB()
sk_clf.fit(X, y);

In [18]:
service.sklearn_CategoricalNB_info(sk_clf)

class_log_prior_:
-0.9162907318741553 -0.5108256237659909 

feature_log_prob_:
[[-0.55961579 -1.25276297 -1.94591015]
 [-1.09861229 -1.5040774  -0.81093022]] [[-0.69314718 -0.69314718]
 [-0.69314718 -0.69314718]] [[-1.09861229 -0.40546511]
 [-0.69314718 -0.69314718]] [[-1.09861229 -0.40546511]
 [-0.28768207 -1.38629436]] 



In [19]:
sk_clf.predict(X_pred)

array(['y1'], dtype='<U2')

##### DecisionTreeClassifier

In [20]:
clf = KNeighborsClassifier()
clf.fit(X, y);

In [21]:
service.KNeighborsClassifier_info(sk_clf, X_pred, 5)

111


In [22]:
clf.predict(X_pred)

Ellipsis

Compare with similar scikit-learn model

In [23]:
sk_clf = sklearn.neighbors.KNeighborsClassifier()
sk_clf.fit(X, y);

In [24]:
service.sklearn_KNeighborsClassifier_info(sk_clf, X_pred, 5)

neighbors_dist:
[1.         1.41421356 1.73205081 1.73205081 1.73205081] 

neighbors_indices:
[8 2 6 1 7] 



In [25]:
sk_clf.predict(X_pred)

array(['y1'], dtype='<U2')

##### KNeighborsClassifier

In [26]:
# TODO

#### data_3x

In [27]:
df = pd.read_csv('data/data_3x.csv')
df.head()

,dataset,x0,x1,x2,y
0,1,0,1,2,1
1,1,1,0,1,1
2,1,0,1,1,0
3,1,0,0,1,1
4,1,0,0,2,1


In [28]:
res_df = pd.DataFrame(
    index=[str(clf) for clf in classifiers],
    columns=df['dataset'].unique()
)

X_pred = pd.DataFrame({'x0': [1], 'x1': [1], 'x2': [1]})
for clf in classifiers:
    for df_i, df_v in df.groupby('dataset'):
        df_v = df_v.drop(columns=['dataset'])
        X, y = df_v.drop(columns='y'), df_v['y']
        clf.fit(X, y)
        y_pred = clf.predict(X_pred)[0]
        res_df.loc[str(clf), df_i] = y_pred

res_df

TypeError: 'ellipsis' object is not subscriptable

#### Check classifiers (scikit-learn)

In [ ]:
import warnings
from sklearn.utils.estimator_checks import estimator_checks_generator

for clf in classifiers:
    for (estimator, check) in estimator_checks_generator(clf):
        try:
            check(estimator)
        except KeyError as e:
            if "encountered an unknown value" in str(e):
                continue
            warnings.warn(str(e))